This approach works selecting k-1 projects for training and testing occurs on the Kth project. 

Along with that, one project is removed by time from the k-1 projects to see if this 1 particular project has effect on the metrics got from evaluating on the Kth project. 

In [1]:
import pandas as pd

In [2]:
msr_data = pd.read_csv('../data/msr/msr_data.csv')
new_data = pd.read_csv('../data/new/new_data.csv')

In [3]:
msr_data.groupby(['project','flaky'])['flaky'].agg({'quantity':'count'})

c:\users\aian\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  """Entry point for launching an IPython kernel.


quantity
project      flaky          
achilles     1            67
alluxio      1             4
ambari       1             4
commons-exec 1             2
dropwizard   1             1
hadoop       0           756
             1           305
handlebars   1             1
hector       1             2
httpcore     0           296
             1             2
jackrabbit   1             8
jimfs        1             7
logback      0            90
             1             3
ninja        0           172
             1            18
okhttp       1            66
oozie        1           857
orbit        1             8
oryx         1            13
spring-boot  1            14
togglz       1            11
wroj4        1            10
zxing        0            88
             1             1

In [4]:
new_data.groupby(['project','flaky'])['flaky'].agg({'quantity':'count'})

c:\users\aian\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  """Entry point for launching an IPython kernel.


quantity
project      flaky          
achilles     0             4
             1             4
activiti     0            32
             1            32
alluxio      0            71
             1           116
ambari       0            52
             1            52
assertjcore  0             1
             1             1
commons-exec 0            45
dubbo        0            19
             1            19
elasticjob   0             3
             1             3
hadoop       0           145
             1           145
handlebars   0             1
             1             1
hector       0            33
             1            33
http-request 0            18
             1            18
httpcore     0            22
             1            22
jackrabbit   0             8
             1             8
logback      0            22
             1            22
ninja        0             1
             1             1
okhttp       0           100
             1           100
orbit        0             7
             1             7
spring-boot  0           160
             1           160
togglz       0             1
             1             1
undertow     0             7
             1             7
websocket    0            23
             1            23
wildfly      0            23
             1            23
wroj4        0            16
             1            16
zxing        0             2
             1             2

In [7]:
new_data.query('project == "spring-boot"')

,token,project,flaky
1,repackage-sources.jar.original forTasks BOOT_V...,spring-boot,1
3,Wrong body: spring-boot-actuator getForObjec...,spring-boot,0
7,next getClass values redisHealthIndicator refr...,spring-boot,0
8,forTasks BOOT_VERSION install -PbootVersion= i...,spring-boot,1
9,assertOutputMatches forTasks BOOT_VERSION -jav...,spring-boot,1
...,...,...,...
1619,checkSpring oldSpringModulesAreNotTransitiveDe...,spring-boot,1
1623,EnvironmentTestUtils myid .id:myid addEnvironm...,spring-boot,0
1627,checkCommonsLogging runBuildForTask commonsLog...,spring-boot,1
1628,result size assertTrue get invoke assertEquals...,spring-boot,0


In [6]:
msr_data.head()

,token,project,flaky
0,call testResourceGetPostParamGet POST MyJsonR...,oozie,1
1,cluster createTable getClusterStatus hcd TEST_...,hadoop,1
2,fail testSetMaxInvalid Assert setDefaultMaxPer...,httpcore,0
3,thinks that it is single rack assertSingleSwi...,hadoop,0
4,hcatUri2 checkCoordAction CoordELFunctions hc...,oozie,1


In [7]:
new_data.head()

,token,project,flaky
0,process testFolder forEachFileInFolder apply g...,wroj4,1
1,repackage-sources.jar.original forTasks BOOT_V...,spring-boot,1
2,add next controller assertFalse testGetActions...,ambari,1
3,Wrong body: spring-boot-actuator getForObjec...,spring-boot,0
4,createTimerJobQuery createTaskQuery createProc...,activiti,1


In [4]:
msr_data['project'].value_counts()

hadoop          1061
oozie            857
httpcore         298
ninja            190
logback           93
zxing             89
achilles          67
okhttp            66
spring-boot       14
oryx              13
togglz            11
wroj4             10
orbit              8
jackrabbit         8
jimfs              7
ambari             4
alluxio            4
commons-exec       2
hector             2
handlebars         1
dropwizard         1
Name: project, dtype: int64

In [5]:
new_data['project'].value_counts()

spring-boot     320
hadoop          290
okhttp          200
alluxio         187
ambari          104
hector           66
activiti         64
wildfly          46
websocket        46
commons-exec     45
logback          44
httpcore         44
dubbo            38
http-request     36
wroj4            32
jackrabbit       16
orbit            14
undertow         14
achilles          8
elasticjob        6
zxing             4
togglz            2
handlebars        2
ninja             2
assertjcore       2
Name: project, dtype: int64

In [14]:
msr_data.shape

(2806, 3)

In [4]:
msr_no_duplicates = msr_data.drop_duplicates()
msr_no_duplicates.shape

(2771, 3)

In [5]:
new_data.shape

(1632, 3)

In [6]:
new_no_duplicates = new_data.drop_duplicates()
new_no_duplicates.shape

(1464, 3)

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, f1_score
import pandas as pd

### Training naive bayes model on MSR data

In [10]:
from sklearn.metrics import accuracy_score

def run_exp(data):
    if data == 'msr':
        exp_data = msr_no_duplicates
    elif data == 'new':
        exp_data = new_no_duplicates
        
    train_projects_list = exp_data['project'].unique()
    print('project list:', train_projects_list, end='\n\n')
    
    for project in train_projects_list:
        print('selected project:', project)

        train_data = exp_data.query(f'project != "{project}"')  
        test_data = exp_data.query(f'project == "{project}"') 
        X_train = train_data['token'].values.astype('U')
        y_train = train_data['flaky']
        X_test = test_data['token'].values.astype('U')
        y_test = test_data['flaky']

        text_clf1 = Pipeline([
                            ('vectorizer', CountVectorizer()),
                            ('model', MultinomialNB())])
        text_clf1.fit(X_train, y_train)
        new_preds = text_clf1.predict(X_test)
        print(classification_report(y_test, new_preds, zero_division=0))
        
        list_of_projects2 = train_data['project'].unique()
        for project_to_drop in list_of_projects2:
            print('dropped project:', project_to_drop)
            dropped_project_data = train_data.query(f'project != "{project_to_drop}"')   
            X_train = dropped_project_data['token'].values.astype('U')
            y_train = dropped_project_data['flaky']            
            
            text_clf1 = Pipeline([
                ('vectorizer', CountVectorizer()),
                ('model', MultinomialNB())])
            text_clf1.fit(X_train, y_train)
            new_preds = text_clf1.predict(X_test)
            print(classification_report(y_test, new_preds, zero_division=0))            

In [11]:
run_exp(data='msr')

project list: ['oozie' 'hadoop' 'httpcore' 'ambari' 'oryx' 'ninja' 'zxing' 'jimfs'
 'logback' 'spring-boot' 'achilles' 'okhttp' 'orbit' 'togglz'
 'commons-exec' 'wroj4' 'alluxio' 'hector' 'jackrabbit' 'handlebars'
 'dropwizard']

selected project: oozie
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.30      0.47       845

    accuracy                           0.30       845
   macro avg       0.50      0.15      0.23       845
weighted avg       1.00      0.30      0.47       845

dropped project: hadoop
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.25      0.40       845

    accuracy                           0.25       845
   macro avg       0.50      0.12      0.20       845
weighted avg       1.00      0.25      0.40       845

dropped project: httpcore
              precision    recall  f1-scor

In [12]:
run_exp(data='new')

project list: ['wroj4' 'spring-boot' 'ambari' 'activiti' 'ninja' 'orbit' 'assertjcore'
 'elasticjob' 'commons-exec' 'websocket' 'alluxio' 'hadoop' 'dubbo'
 'okhttp' 'httpcore' 'hector' 'logback' 'wildfly' 'togglz' 'zxing'
 'http-request' 'jackrabbit' 'achilles' 'undertow' 'handlebars']

selected project: wroj4
              precision    recall  f1-score   support

           0       0.78      0.88      0.82        16
           1       0.86      0.75      0.80        16

    accuracy                           0.81        32
   macro avg       0.82      0.81      0.81        32
weighted avg       0.82      0.81      0.81        32

dropped project: spring-boot
              precision    recall  f1-score   support

           0       0.75      0.56      0.64        16
           1       0.65      0.81      0.72        16

    accuracy                           0.69        32
   macro avg       0.70      0.69      0.68        32
weighted avg       0.70      0.69      0.68        32

dropp